In [1]:
random_state = 129763 # In case I need it

try:
    import os
    print(os.getcwd())
    import tensorflow as tf
    print( tf.constant( 'Hello from TensorFlow ' + tf.__version__ ) )
    import pandas as pd
    import numpy as np
    from numpy import array
    from keras.models import Sequential
    from keras.layers import LSTM
    from keras.layers import Dense

    print("✔️ Successfully imported all packages")
except Exception as ex:
    print("❌ Could not import one or more libraries:", ex)

c:\Users\thomas.henno\Desktop\GitRepos\course-machine-learning\week-four\AS14
tf.Tensor(b'Hello from TensorFlow 2.1.0', shape=(), dtype=string)
✔️ Successfully imported all packages
Using TensorFlow backend.


In [2]:
df = pd.read_excel("Dataset.xlsx", skiprows=2, header=None)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,673,674,675,676,677,678,679,680,681,682
0,2521.234,1554.951,1760.839,1874.076,1770.127,2167.416,2250.602,1567.658,2382.911,1786.066,...,1329.016,1056.066,1736.332,1544.154,1555.864,1702.201,1854.153,1646.496,1520.763,1368.026
1,1944.512,1566.987,1224.113,1500.748,3558.259,2831.670,2078.168,2216.085,1804.972,2051.166,...,5592.715,1927.505,2702.186,1364.702,2801.333,2110.587,2153.170,1852.452,1991.153,2549.164
2,2045.401,1691.263,4474.665,2021.804,5016.791,1812.510,3736.011,2855.442,2085.921,1721.896,...,2012.310,1700.974,1708.750,2220.219,1324.229,2571.308,1900.011,2130.460,1793.301,1772.942
3,2165.915,2214.965,2395.277,1909.094,1653.703,2003.161,1731.560,1907.299,1801.705,1906.117,...,1215.273,1127.290,2038.196,1871.442,1468.785,1840.928,1682.228,2194.603,2787.753,1712.142
4,2745.535,2794.596,2250.799,2202.103,1552.819,2111.839,1948.280,1637.539,1694.834,1754.551,...,1832.339,1548.872,1293.723,2272.632,1986.716,1863.857,1858.556,2018.394,1623.119,1658.063


In [3]:
df = df.aggregate("mean", axis="rows")

In [4]:
sequence = [x for x in df.values.tolist()]
print(sequence)
del df

[2178.962833333333, 2140.8261666666663, 2029.3010833333335, 1878.8448611111114, 2105.957083333333, 1998.5725555555557, 2226.943861111111, 1938.2611944444445, 1807.1166666666666, 1781.9076111111108, 1794.381, 1867.543138888889, 1881.5988888888892, 1788.7648333333334, 1769.7328888888887, 1778.89275, 1750.3532777777777, 1744.098472222222, 1605.8494722222222, 1602.4786388888888, 1610.041694444445, 1582.7103333333334, 1754.1105, 1821.1048333333335, 2056.823611111111, 1912.785083333334, 1761.8987499999998, 1809.4357499999996, 2006.3628055555555, 1858.2195555555554, 1952.6886666666662, 2025.8715000000002, 1913.2763055555556, 1765.9903888888887, 1763.3245555555554, 1827.8902777777778, 1909.5682499999998, 1799.694388888889, 1741.173055555556, 1827.0989444444447, 1816.642388888889, 1829.5460277777775, 1801.1698333333334, 1763.7866111111111, 1595.841666666667, 1623.936027777778, 1614.531388888889, 1865.5651388888884, 1999.4232777777777, 2141.1095, 2113.6754999999994, 1858.3847777777778, 1857.6436

In [5]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [6]:
n_steps = 48

# Split into samples
X, y = split_sequence(sequence, n_steps)

X_train = X[0:504]
y_train = y[0:504]

X_test = X[504:]
y_test = y[504:]

print(type(X_train), len(X_train))
print(type(X_test), len(X_test))
print(type(y_train), len(y_train))
print(type(y_test), len(y_test))

<class 'numpy.ndarray'> 504
<class 'numpy.ndarray'> 131
<class 'numpy.ndarray'> 504
<class 'numpy.ndarray'> 131


In [7]:
n_features = 1
# reshape from [samples, timesteps] into [samples, timesteps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
X_train.shape

(504, 48, 1)

In [12]:
# Make LSTM model
model = Sequential()
model.add(LSTM(75, activation="relu", input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mse")

# Fit model
model.fit(X_train, y_train, epochs=200, verbose=1)#, batch_size=5)

Epoch 1/200
504/504 [==============================] - 1s 1ms/step - loss: 5511714.8175
Epoch 2/200
504/504 [==============================] - 0s 752us/step - loss: 3975890.5238
Epoch 3/200
504/504 [==============================] - 0s 726us/step - loss: 2562058.5972
Epoch 4/200
504/504 [==============================] - 0s 730us/step - loss: 4244261.8016
Epoch 5/200
504/504 [==============================] - 0s 730us/step - loss: 2680563.7063
Epoch 6/200
504/504 [==============================] - 0s 722us/step - loss: 851047.7540
Epoch 7/200
504/504 [==============================] - 0s 744us/step - loss: 767227.4444
Epoch 8/200
504/504 [==============================] - 0s 760us/step - loss: 721021.3770
Epoch 9/200
504/504 [==============================] - 0s 746us/step - loss: 602225.7604
Epoch 10/200
504/504 [==============================] - 0s 740us/step - loss: 544183.3938
Epoch 11/200
504/504 [==============================] - 0s 728us/step - loss: 592472.4945
Epoch 12/200
504

In [14]:
# Prediction
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
y_pred = model.predict(X_test, verbose=0)
# print(y_pred)

In [29]:
total_error = 0
largest_error = None
min_error = None
for real, pred in zip(y_test, y_pred):
    print("Expected: %d Got: %d" % (real, pred[0]), end=" | ")
    error = int(pred[0]) - int(real)
    print(error)
    if error < 0:
        error = error*-1
    total_error += error
    if largest_error == None and min_error == None:
        largest_error, min_error = error, error
    if error > largest_error:
        largest_error = error
    elif error < min_error:
        min_error = error

print("")
print("Total mean error:", str(int(total_error/len(y_test))))
print("Largest error:", largest_error)
print("Smallest error:", min_error)

Expected: 2120 Got: 1812 | -308
Expected: 1831 Got: 2016 | 185
Expected: 1961 Got: 1914 | -47
Expected: 1813 Got: 1825 | 12
Expected: 1809 Got: 2003 | 194
Expected: 1974 Got: 1811 | -163
Expected: 2005 Got: 1952 | -53
Expected: 1948 Got: 1922 | -26
Expected: 1921 Got: 1922 | 1
Expected: 1858 Got: 1940 | 82
Expected: 1852 Got: 1889 | 37
Expected: 1918 Got: 2085 | 167
Expected: 1999 Got: 1996 | -3
Expected: 1750 Got: 2075 | 325
Expected: 1845 Got: 1914 | 69
Expected: 1807 Got: 1846 | 39
Expected: 1752 Got: 1815 | 63
Expected: 1782 Got: 1834 | 52
Expected: 1839 Got: 1798 | -41
Expected: 1709 Got: 1845 | 136
Expected: 1853 Got: 1876 | 23
Expected: 1798 Got: 1777 | -21
Expected: 1871 Got: 1737 | -134
Expected: 2038 Got: 1774 | -264
Expected: 2287 Got: 1838 | -449
Expected: 2304 Got: 1932 | -372
Expected: 2231 Got: 2009 | -222
Expected: 2056 Got: 1983 | -73
Expected: 2066 Got: 1952 | -114
Expected: 2043 Got: 1964 | -79
Expected: 2063 Got: 1959 | -104
Expected: 2068 Got: 1914 | -154
Expected: